In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import folium
from folium import Choropleth
from folium import GeoJson
import geopandas as gpd

import plotly.express as px

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import os

In [ ]:
df = pd.read_csv('../bin/data_preprocess.csv')

In [ ]:
mapa_provincias = gpd.read_file('../bin/limites provinciales/recintos_provinciales_inspire_peninbal_etrs89.shp')

In [ ]:
coches_por_provincia = df['id_provincia'].value_counts().reset_index()
coches_por_provincia.columns = ['provincia', 'cantidad']

In [ ]:
def extraer_segundo_nombre(provincia):
    if '/' in provincia:
        return provincia.split('/')[1].strip()
    return provincia

# Aplicar la función al GeoDataFrame para modificar la columna de las provincias
mapa_provincias['NAMEUNIT'] = mapa_provincias['NAMEUNIT'].apply(extraer_segundo_nombre)

In [ ]:
mapa_provincias['NAMEUNIT'] = mapa_provincias['NAMEUNIT'].replace({
    'A Coruña': 'La Coruña',
    'Bizkaia': 'Vizcaya',
    'Ourense': 'Orense',
    'Illes Balears': 'Islas Baleares',
    'Gipuzkoa': 'Guipúzcoa'
})

In [ ]:
nombres_listado = set(df['id_provincia'].unique())
nombres_mapa_actualizados = set(mapa_provincias['NAMEUNIT'].unique())
nombres_unicos_actualizados = nombres_mapa_actualizados - nombres_listado

nombres_unicos_actualizados

In [ ]:
mapa_provincias = mapa_provincias.merge(coches_por_provincia, left_on='NAMEUNIT', right_on='provincia', how='left')

In [ ]:
plt.figure(figsize=(20, 15))
mapa_provincias.plot(column='cantidad', cmap='Oranges', legend=True, edgecolor='black')
plt.title('Cantidad de Coches por Provincia en España')
plt.show()

In [ ]:
mapa_provincias_geojson = mapa_provincias.to_crs("EPSG:4326").to_json()
cantidad_dict = coches_por_provincia.set_index('provincia')['cantidad'].to_dict()

mapa_provincias['cantidad'] = mapa_provincias['NAMEUNIT'].map(cantidad_dict).fillna(0).astype(int)
mapa_provincias['cantidad_log'] = np.log1p(mapa_provincias['cantidad'])
mapa_provincias_geojson = gpd.GeoDataFrame(mapa_provincias).to_crs("EPSG:4326").to_json()

mapa = folium.Map(location=[40.4168, -3.7038], zoom_start=6)

Choropleth(
    geo_data=mapa_provincias_geojson,
    data=mapa_provincias,
    columns=['provincia', 'cantidad_log'],
    key_on='feature.properties.NAMEUNIT',
    fill_color='Oranges',
    fill_opacity=1,
    line_opacity=0.2,
    legend_name='Cantidad de Coches',
    nan_fill_color='white'
).add_to(mapa)

GeoJson(
    data=mapa_provincias,
    tooltip=folium.GeoJsonTooltip(
        fields=['NAMEUNIT', 'cantidad'],
        aliases=['Provincia:', 'Cantidad de Coches:']
    )
).add_to(mapa)


#Guardamos el mapa en html y luego en png
#El proceso tiene varios pasos

mapa.save("../bin/mapa_cantidad.html")

ruta_salida = "../bin/mapa_cantidad.png"

os.makedirs("../bin", exist_ok=True)

options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Ejecutar sin abrir navegador
options.add_argument("--window-size=1024x768")  # Tamaño de ventana
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get(f"file://{os.path.abspath('../bin/mapa_cantidad.html')}")

driver.save_screenshot(ruta_salida)

driver.quit()

mapa

In [ ]:
precio_medio_por_provincia = df.groupby('id_provincia')['precio_contado'].mean().reset_index()
precio_medio_por_provincia.columns = ['provincia', 'precio_medio']
#precio_medio_por_provincia['precio_medio'] = precio_medio_por_provincia['precio_medio'].round(2)

precio_medio_dict = precio_medio_por_provincia.set_index('provincia')['precio_medio'].to_dict()

# Añadir el precio medio al GeoDataFrame
mapa_provincias['precio_medio'] = np.round(mapa_provincias['NAMEUNIT'].map(precio_medio_dict).fillna(0), 2)

mapa = folium.Map(location=[40.4168, -3.7038], zoom_start=6)

Choropleth(
    geo_data=mapa_provincias_geojson,
    data=mapa_provincias,
    columns=['NAMEUNIT', 'precio_medio'],
    key_on='feature.properties.NAMEUNIT',
    fill_color='Oranges',
    fill_opacity=1,
    line_opacity=0.2,
    legend_name='Precio Medio por Provincia',
    nan_fill_color='white'
).add_to(mapa)

GeoJson(
    data=mapa_provincias,
    tooltip=folium.GeoJsonTooltip(
        fields=['NAMEUNIT', 'precio_medio'],
        aliases=['Provincia:', 'Precio Medio:']
    )
).add_to(mapa)

mapa.save("../bin/mapa_precio.html")

ruta_salida = "../bin/mapa_precio.png"

os.makedirs("../bin", exist_ok=True)

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--window-size=1024x768")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get(f"file://{os.path.abspath('../bin/mapa_precio.html')}")

driver.save_screenshot(ruta_salida)

driver.quit()

mapa